In [1]:
import sys
sys.path.append('..')
sys.path.append('/Users/jh7685/Documents/Projects/pysurfer')
from pysurfer.mgz_helper import map_values_as_mgz
#import freeview_helper as pyview
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
from scipy.stats import f_oneway
import warnings
from itertools import product
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

In [2]:
nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
#freesurfer_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'

# Load in Beta values

In [3]:
subj='subj01'
hemi='lh'
betas = os.path.join(nsd_dir, 'nsddata_betas', 'ppdata', f'{subj}', 'nativesurface', 'nsdsyntheticbetas_fithrf_GLMdenoise_RR', f'{hemi}.betas_nsdsynthetic.hdf5')
design_mat=os.path.join(nsd_dir, 'nsddata', 'experiments', 'nsdsynthetic', 'nsdsynthetic_expdesign.mat')
task_keys=['fixation_task','memory_task']
stim_info=os.path.join(nsd_dir, 'nsdsyn_stim_description.csv')
task_average=True

roi_vals = [prep.convert_between_roi_num_and_vareas("V1"), [1,2,3,4,5]]
roi_path = [os.path.join(nsd_dir, 'nsddata', 'freesurfer', f'{subj}', 'label', f'{hemi}.prf-{roi_file}.mgz') for roi_file in ["visualrois", "eccrois"]]
mask, roi_dict = prep.load_common_mask_and_rois(roi_path, roi_vals)

In [4]:
betas_df = sfm.get_whole_brain_betas(betas_path=betas, design_mat_path=design_mat,
                          stim_info_path=stim_info,
                          task_keys=task_keys, task_average=task_average,
                          x_axis='voxel', y_axis='stim_idx', long_format=True)


In [5]:
n = 0
one_voxel_betas_df = betas_df.query('voxel == @n')
sfm.sf_one_way_anova(one_voxel_betas_df, 'freq_lvl', test_unique=[0,1,2,3,4,5])

TypeError: sf_one_way_anova() missing 1 required positional argument: 'values'

In [8]:
one_voxel_betas_df.query('freq_lvl == 0')

,stim_idx,image_idx,names,w_r,w_a,phase,class_idx,freq_lvl,voxel,task,betas
0,0,104,pinwheel,0.0,6.0,0.000000,0,0,0,avg,1.321667
227021,1,105,pinwheel,0.0,6.0,1.570796,0,0,0,avg,2.753333
454042,2,106,pinwheel,0.0,6.0,3.141593,0,0,0,avg,1.098333
681063,3,107,pinwheel,0.0,6.0,4.712389,0,0,0,avg,1.440000
5448504,24,128,forward spiral,4.0,4.0,0.000000,6,0,0,avg,0.658333
5675525,25,129,forward spiral,4.0,4.0,1.570796,6,0,0,avg,0.933333
5902546,26,130,forward spiral,4.0,4.0,3.141593,6,0,0,avg,0.831667
6129567,27,131,forward spiral,4.0,4.0,4.712389,6,0,0,avg,1.463333
10897008,48,152,annulus,6.0,0.0,0.000000,12,0,0,avg,0.796667
11124029,49,153,annulus,6.0,0.0,1.570796,12,0,0,avg,0.338333


In [75]:
names_list = betas_df.names.unique().tolist()[:4]
F, p, identifiers = sfm.sf_multiple_one_way_anova(betas_df.query('names in @names_list'), to_test='freq_lvl', values='betas', on='voxel', identifier_list=['names','phase'], test_unique=None)



In [100]:
template_path=os.path.join(nsd_dir, 'nsddata', 'freesurfer',f'{subj}', 'label', f'{hemi}.prfeccentricity.mgz')
mgz_path=os.path.join(output_dir, 'sfp_anova',  'brain_maps', 'nsdsyn', f'{hemi}.sub-{subj}_stat-anova_value-F.mgz')
os.makedirs(os.path.dirname(mgz_path), exist_ok=True)
print(os.path.dirname(mgz_path))

/Volumes/server/Projects/sfp_nsd/derivatives/sfp_anova/brain_maps/nsdsyn


In [101]:
test = map_values_as_mgz(template_path, F, save_path=mgz_path)

In [104]:
fs_subj_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'
sn='subj01'
surf='inflated'
overlay=f'sub-{subj}_stat-anova_value-F.mgz'
overlay_dir=os.path.dirname(mgz_path)

lh_obj = pyview.PyFreeview(fs_subj_dir,
                           sn, hemi,
                           surf=surf,
                           overlay=overlay,
                           overlay_dir=overlay_dir)
lh_obj.run(**{'overlay_custom': None})

return code: 1
Unable to load color scale from  "None"
Unable to load color scale from  "None"



CompletedProcess(args='freeview -f /Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer/subj01/surf/lh.inflated:curvature_method=binary::overlay=/Volumes/server/Projects/sfp_nsd/derivatives/sfp_anova/brain_maps/nsdsyn/lh.sub-subj01_stat-anova_value-F.mgz:overlay_color=heat:overlay_opacity=1:overlay_custom=None:', returncode=1, stdout=b'Unable to load color scale from  "None"\nUnable to load color scale from  "None"\n', stderr=b'QIODevice::read (QFile, "None"): device not open\nQIODevice::read (QFile, "None"): device not open\n2023-10-26 17:32:06.428 freeview[64357:2928579] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit\n')